In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from spacy.cli.train import train
import spacy

2023-05-02 12:00:27.434476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
train("config/config_compact_bert.cfg",
      output_path='sig_ner_bert2_model',
      overrides={"paths.train": "/Users/royashcenazi/Studies/data/train_docs.spacy", 
                 "paths.dev": "/Users/royashcenazi/Studies/data/test_docs.spacy",
                 "training.max_epochs": 16})

# !python3 -m spacy train config/config_bert.cfg -o sig_bert_model --paths.train train_docs.spacy --paths.dev test_docs.spacy --training.max_epochs 8


ℹ Saving to output directory: sig_ner_bert2_model
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================


Some weights of the model checkpoint at nlpie/compact-biobert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at nlpie/compact-biobert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']

✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0       34600.13   1233.77    0.05    0.04    0.06    0.00
  3     200      595313.27  90023.28   66.81   76.86   59.09    0.67
  6     400       31489.43  35282.71   82.12   83.19   81.08    0.82
 10     600       14314.17  30800.98   86.06   84.78   87.38    0.86
 13     800        3639.77  29515.45   87.52   87.77   87.27    0.88
✔ Saved pipeline to output directory
sig_ner_bert2_model/model-last


In [20]:
trained_nlp = spacy.load('../../parsigs-models/sig_bert_model/model-best/')

##### first outcome! 

In [8]:
inp = 'TAKE 1 TABLET  (150 MCG TOTAL) BY MOUTH two times DAILY for one week by the way patient id is 303023 and take 2 aderol every day'.lower()
[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tablet, 'Form'),
 (150 mcg, 'Strength'),
 (two times daily, 'Frequency'),
 (for one week, 'Duration'),
 (2, 'Dosage'),
 (aderol, 'Drug'),
 (every day, 'Frequency')]

In [9]:
inp = 'Take 2 tablets of aderol in the morning every week'.lower()
[(e, e.label_) for e in trained_nlp(inp).ents]

[(2, 'Dosage'),
 (tablets, 'Form'),
 (aderol, 'Drug'),
 (morning every week, 'Frequency')]

In [10]:
inp = "Take 2 capsules qid of amoxicillin 500mg"
[(e, e.label_) for e in trained_nlp(inp).ents]

[(2, 'Dosage'),
 (capsules, 'Form'),
 (qid, 'Frequency'),
 (amoxicillin, 'Drug'),
 (500mg, 'Strength')]

 We can even parse multuple dosing instructions

In [11]:
inp = 'inhale 3 puffs of albuterol 2 times a day for one week' \
      ' then 1 puff of albuterol every day for 2 months'

print([(e, e.label_) for e in trained_nlp(inp).ents])

[(3, 'Dosage'), (puffs, 'Form'), (albuterol, 'Drug'), (2 times a day, 'Frequency'), (for one week, 'Duration'), (1, 'Dosage'), (puff, 'Form'), (albuterol, 'Drug'), (every day, 'Frequency'), (for 2 months, 'Duration')]


In [20]:
inp = 'TAKE 1 TABLET (20 MG) BY MOUTH every 2 months'.lower()
print([(e, e.label_) for e in trained_nlp(inp).ents])

[(tablet, 'Form'), (20 mg, 'Strength')]


 There are many things to improve, one example is the frequency that can and should be parsed to an interval type and amoubt (e.g type=day, amount=2)

Also, the model only recognizes encountered features, so inputting to the model a new sentence, will not be parsed correctly. 
When we will use the pre-trained Bert model, this issue should be resolved

In [11]:
inp = 'Take 1 tablet of ibuprofen 3 times a day'


[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tablet, 'Form'),
 (ibuprofen, 'Drug'),
 (3 times a day, 'Dosage')]

In [13]:
inp = '1 TAB Q7d'.lower()

[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'), (q7d, 'Frequency')]

In [3]:
inp = 'Take 1 tab of Minesse for 24 days then skip 4 days'

[(e, e.label_) for e in trained_nlp(inp).ents]

[(1, 'Dosage'),
 (tab of, 'Form'),
 (for 24 days, 'Duration'),
 (4 days, 'Duration')]

In [2]:
trial = spacy.load("en_parsigs")

In [8]:
inp = 'Take 1 tab of Minesse for 24 days then skip 4 days'

[(e, e.label_) for e in 
(inp).ents]


[(1, 'Dosage'),
 (tab of, 'Form'),
 (for 24 days, 'Duration'),
 (4 days, 'Duration')]